In [68]:
import os
import numpy as np
import pandas as pd

PATH = '/home/seunghan9613/S-D-Mamba/results'
os.chdir(PATH)

In [69]:
settings = os.listdir()
print(len(settings))

1585


In [70]:
def get_result(DATASET):
    settings_dataset = [x for x in settings if DATASET in x]
    fnames = [os.path.join(x,'metrics.npy') for x in settings_dataset]
    fnames = [x for x in fnames if os.path.isfile(x)]
    
    result_dict = dict()
    for f in fnames:
        mae, mse, rmse, mape, mspe = np.load(f)
        result_dict[f] = mse
    
    df = pd.DataFrame(pd.Series(result_dict).sort_values())
 
    df.reset_index(inplace=True)
    df['data'] = df['index'].apply(lambda x:x.split('_')[0])
    df['L'] = df['index'].apply(lambda x:int(x.split('_')[2]))
    df['reg'] = df['index'].apply(lambda x:x.split('_')[5])
    df['lamb'] = df['index'].apply(lambda x:x.split('_0')[1].replace('metrics.npy','').replace('/',''))
    
    lambda_list = []
    k1_list = []
    mask_list = []
    conv_list = []
    wo_TD1_list = []
    wo_TD2_list = []

    for x in df['index']:
        if 'wo_conv' in x:
            conv_list.append(0)
        else:
            conv_list.append(1)
        if 'wo_TD1' in x:
            wo_TD1_list.append(0)
        else:
            wo_TD1_list.append(1)
        if 'wo_TD2' in x:
            wo_TD2_list.append(0)
        else:
            wo_TD2_list.append(1)
            
    for x in df['lamb']:
        if 'lambda' in x:
            lamb = x.split('lambda')[1].split('_')[0]
        else:
            lamb = 0
        
        if 'mask' in x:
            mask = x.split('mask')[1].split('_')[0]
        else:
            mask = 0
        if 'lambda' in x:
            lamb = x.split('lambda')[1].split('_')[0]
        else:
            lamb = 0
        
        if 'mask' in x:
            mask = x.split('mask')[1].split('_')[0]
        else:
            mask = 0
        lambda_list.append(float(lamb))
        mask_list.append(float(mask))
    df['lamb'] = lambda_list
    df['mask'] = mask_list
    df['conv'] = conv_list
    df['TD1'] = wo_TD1_list
    df['TD2'] = wo_TD2_list
    
    df.drop('index',axis=1,inplace=True)
    df.columns = ['mse','data','L','reg','lamb','mask','conv','TD1','TD2']
    df = df[['data','L','reg','lamb','mask','conv','TD1','TD2','mse']]
    df = df.pivot_table(index=['data', 'reg','lamb','mask','conv','TD1','TD2'], columns='L', values='mse').reset_index()
    #================================================================================#
    df = df[(df['reg']!='reg')&(df['reg']!='reg2')]
    #================================================================================#
    return df


# SOLAR

In [71]:
df = get_result(DATASET='solar')
df['avg'] = df.iloc[:,-4:].mean(axis=1)
#df[(df['reg']=='reg3')&(df['lamb']==0)&(df['conv']==0)&(df['TD1']==1)&(df['TD2']==1)].round(3)
#df[(df['reg']=='reg4')&(df['lamb']==0)&(df['conv']==0)&(df['TD1']==1)&(df['TD2']==1)].round(3)
#df

In [72]:
solar = df.groupby(['reg','mask','conv','TD1','TD2']).min()[[96,192,336,720]]
solar.loc['base'] = [0.205,0.237,0.258,0.260]
solar.loc['repr'] = [0.207,0.240,0.262,0.267]

solar['avg'] = solar.mean(axis=1)
solar.sort_values('avg').round(3)


L,96,192,336,720,avg
"(reg4, 0.0, 0, 1, 0)",0.211,0.241,0.264,NaN,0.239
base,0.205,0.237,0.258,0.260,0.240
"(reg3, 0.0, 0, 1, 1)",0.206,0.239,0.259,0.263,0.242
"(reg3, 0.0, 0, 0, 1)",0.208,0.239,0.260,0.265,0.243
"(reg4, 0.0, 0, 1, 1)",0.207,0.240,0.261,0.265,0.243
"(reg3, 0.0, 0, 1, 0)",0.208,0.240,0.261,0.266,0.244
"(reg3, 0.0, 1, 1, 1)",0.208,0.241,0.262,0.266,0.244
repr,0.207,0.240,0.262,0.267,0.244
"(reg4, 0.0, 0, 0, 1)",0.210,0.241,0.261,0.266,0.244
"(reg4, 0.0, 1, 1, 1)",0.210,0.243,0.263,0.266,0.246


# ETTh1:끝

In [73]:
df = get_result(DATASET='ETTh1')
#df['avg'] = df.iloc[:,-4:].mean(axis=1)
#df[(df['reg']=='reg3')&(df['lamb']==0)&(df['conv']==0)&(df['TD1']==1)&(df['TD2']==1)].round(3)
#df[(df['reg']=='reg4')&(df['lamb']==0)&(df['conv']==0)&(df['TD1']==1)&(df['TD2']==1)].round(3)
df

L,data,reg,lamb,mask,conv,TD1,TD2,96,192,336,720
17,ETTh1,reg3,0.000,0.0,0,0,1,0.385077,0.434284,0.476952,0.477833
18,ETTh1,reg3,0.000,0.0,0,1,0,0.385802,0.435356,0.481337,0.482736
19,ETTh1,reg3,0.000,0.0,0,1,1,0.383067,0.433165,0.477481,0.476243
20,ETTh1,reg3,0.000,0.0,1,1,1,0.387338,0.441882,0.488259,0.504139
21,ETTh1,reg3,0.001,0.0,0,0,1,0.384745,0.433512,0.477542,0.477429
22,ETTh1,reg3,0.001,0.0,0,1,0,0.385327,0.434873,0.481665,0.483831
23,ETTh1,reg3,0.001,0.0,0,1,1,0.382553,0.432631,0.477787,0.476942
24,ETTh1,reg3,0.001,0.0,1,1,1,0.385426,0.438625,0.485168,0.498114
25,ETTh1,reg3,0.010,0.0,0,0,1,0.384760,0.433494,0.477588,0.477441
26,ETTh1,reg3,0.010,0.0,0,1,0,0.385307,0.434858,0.481688,0.483787


In [54]:
etth1 = df.groupby(['reg','mask','conv','TD1','TD2']).min()[[96,192,336,720]]
etth1.loc['base'] = [0.386,0.443,0.489,0.502]
etth1.loc['repr'] = [0.385,0.445,0.490,0.506]

etth1['avg'] = etth1.mean(axis=1)
etth1.sort_values('avg').round(3)

L,96,192,336,720,avg
"(reg3, 0.0, 0, 1, 1)",0.383,0.433,0.477,0.476,0.442
"(reg3, 0.0, 0, 0, 1)",0.385,0.433,0.477,0.477,0.443
"(reg4, 0.0, 0, 1, 1)",0.383,0.433,0.479,0.482,0.444
"(reg4, 0.0, 0, 0, 1)",0.386,0.434,0.479,0.481,0.445
"(reg3, 0.0, 0, 1, 0)",0.385,0.435,0.481,0.483,0.446
"(reg4, 0.0, 0, 1, 0)",0.386,0.436,0.483,0.487,0.448
"(reg3, 0.0, 1, 1, 1)",0.384,0.436,0.483,0.494,0.449
"(reg4, 0.0, 1, 1, 1)",0.384,0.436,0.483,0.496,0.450
base,0.386,0.443,0.489,0.502,0.455
repr,0.385,0.445,0.490,0.506,0.456


In [55]:
df.groupby(['reg','mask','conv','TD1','TD2']).min()[[96,192,336,720]]

L                             96       192       336       720
reg  mask conv TD1 TD2                                        
reg3 0.0  0    0   1    0.384745  0.433494  0.476952  0.477429
               1   0    0.385290  0.434858  0.481337  0.482736
                   1    0.382514  0.432604  0.477481  0.476243
          1    1   1    0.384135  0.435505  0.483341  0.494436
reg4 0.0  0    0   1    0.385518  0.434038  0.478738  0.481348
               1   0    0.385869  0.435570  0.482763  0.486880
                   1    0.382809  0.432981  0.478761  0.482397
          1    1   1    0.383920  0.435681  0.483302  0.496324

# ETTh2:끝

In [74]:
df = get_result(DATASET='ETTh2')
df['avg'] = df.iloc[:,-4:].mean(axis=1)
##df[(df['reg']=='reg3')&(df['lamb']==0)&(df['conv']==0)&(df['TD1']==1)&(df['TD2']==1)].round(3)
#df[(df['reg']=='reg4')&(df['lamb']==0)&(df['conv']==0)&(df['TD1']==1)&(df['TD2']==1)].round(3)
df

L,data,reg,lamb,mask,conv,TD1,TD2,96,192,336,720,avg
27,ETTh2,reg3,0.000,0.0,0,0,1,0.303011,0.380066,0.430893,0.435796,0.387442
28,ETTh2,reg3,0.000,0.0,0,1,0,0.302679,0.378845,0.429342,0.435035,0.386475
29,ETTh2,reg3,0.000,0.0,0,1,1,0.299404,0.375304,0.423466,0.431809,0.382496
30,ETTh2,reg3,0.000,0.0,1,1,1,0.295924,0.378772,0.424342,0.434238,0.383319
31,ETTh2,reg3,0.001,0.0,0,0,1,0.303013,0.380177,0.430876,0.435540,0.387402
32,ETTh2,reg3,0.001,0.0,0,1,0,0.302493,0.378946,0.429246,0.434500,0.386296
33,ETTh2,reg3,0.001,0.0,0,1,1,0.299265,0.375332,0.423482,0.431589,0.382417
34,ETTh2,reg3,0.001,0.0,1,1,1,0.296264,0.378721,0.424368,0.433717,0.383267
35,ETTh2,reg3,0.010,0.0,0,0,1,0.302628,0.380289,0.430774,0.435284,0.387244
36,ETTh2,reg3,0.010,0.0,0,1,0,0.302375,0.379077,0.429192,0.434351,0.386249


In [33]:
etth2 = df.groupby(['reg','mask','conv','TD1','TD2']).min()[[96,192,336,720]]
etth2.loc['base'] = [0.296,0.376,0.424,0.426]
etth2.loc['repr'] = [0.297,0.378,0.425,0.432]
etth2['avg'] = etth2.mean(axis=1)
etth2.sort_values('avg').round(3)

L,96,192,336,720,avg
"(reg3, 0.0, 1, 1, 1)",0.296,0.377,0.424,0.424,0.380
base,0.296,0.376,0.424,0.426,0.380
"(reg4, 0.0, 1, 1, 1)",0.297,0.378,0.425,0.424,0.381
"(reg4, 0.0, 0, 1, 1)",0.298,0.375,0.424,0.430,0.382
"(reg3, 0.0, 0, 1, 1)",0.299,0.375,0.423,0.431,0.382
repr,0.297,0.378,0.425,0.432,0.383
"(reg4, 0.0, 0, 1, 0)",0.301,0.379,0.430,0.433,0.386
"(reg3, 0.0, 0, 1, 0)",0.302,0.379,0.429,0.434,0.386
"(reg3, 0.0, 0, 0, 1)",0.303,0.380,0.431,0.435,0.387
"(reg4, 0.0, 0, 0, 1)",0.302,0.382,0.432,0.434,0.387


# ETTm1:끝

In [57]:
df = get_result(DATASET='ETTm1')
#df['avg'] = df.iloc[:,-4:].mean(axis=1)
#df[(df['reg']=='reg3')&(df['lamb']==0)&(df['conv']==0)&(df['TD1']==1)&(df['TD2']==1)].round(3)
#df[(df['reg']=='reg4')&(df['lamb']==0)&(df['conv']==0)&(df['TD1']==1)&(df['TD2']==1)].round(3)
df

L,data,reg,lamb,mask,conv,TD1,TD2,96,192,336,720
22,ETTm1,reg3,0.000,0.0,0,0,1,0.329432,0.378779,0.413313,0.475297
23,ETTm1,reg3,0.000,0.0,0,1,1,0.335776,0.375879,0.411619,0.475082
24,ETTm1,reg3,0.000,0.0,1,1,1,0.340110,0.380959,0.413125,0.476076
25,ETTm1,reg3,0.001,0.0,0,0,1,0.326708,0.377912,0.412152,0.474471
26,ETTm1,reg3,0.001,0.0,0,1,1,0.326254,0.374792,0.409671,0.472952
27,ETTm1,reg3,0.001,0.0,1,1,1,0.335884,0.377068,0.409150,0.473309
28,ETTm1,reg3,0.010,0.0,0,0,1,0.326937,0.378007,0.412186,0.474460
29,ETTm1,reg3,0.010,0.0,0,1,1,0.326043,0.374799,0.409674,0.472934
30,ETTm1,reg3,0.010,0.0,1,1,1,0.329043,0.374556,0.409043,0.471893
31,ETTm1,reg3,0.100,0.0,0,0,1,0.326900,0.377999,0.412174,0.474338


In [18]:
ettm1 = df.groupby(['reg','mask','conv','TD1','TD2']).min()[[96,192,336,720]]
ettm1.loc['base'] = [0.333,0.376,0.408,0.475]
ettm1.loc['repr'] = [0.331,0.378,0.410,0.474]

ettm1['avg'] = ettm1.mean(axis=1)
ettm1.sort_values('avg').round(3)

L,96,192,336,720,avg
"(reg4, 0.0, 0, 0, 1)",0.329,0.376,NaN,NaN,0.353
"(reg4, 0.0, 1, 1, 1)",0.325,0.372,0.404,0.467,0.392
"(reg4, 0.0, 0, 1, 1)",0.326,0.374,0.409,0.473,0.395
"(reg3, 0.0, 0, 1, 1)",0.326,0.375,0.410,0.473,0.396
"(reg3, 0.0, 1, 1, 1)",0.329,0.375,0.409,0.472,0.396
"(reg3, 0.0, 0, 0, 1)",0.327,0.378,0.412,0.474,0.398
base,0.333,0.376,0.408,0.475,0.398
repr,0.331,0.378,0.410,0.474,0.398


# ETTm2:끝

In [58]:
df = get_result(DATASET='ETTm2')
#df['avg'] = df.iloc[:,-4:].mean(axis=1)
#df[(df['reg']=='reg3')&(df['lamb']==0)&(df['conv']==0)&(df['TD1']==1)&(df['TD2']==1)].round(3)
#df[(df['reg']=='reg4')&(df['lamb']==0)&(df['conv']==0)&(df['TD1']==1)&(df['TD2']==1)].round(3)
df

L,data,reg,lamb,mask,conv,TD1,TD2,96,192,336,720
22,ETTm2,reg3,0.000,0.0,0,0,1,0.179924,0.247936,0.310903,0.404929
23,ETTm2,reg3,0.000,0.0,0,1,1,0.180878,0.246762,0.309604,0.404334
24,ETTm2,reg3,0.000,0.0,1,1,1,0.182222,0.247787,0.312041,0.412966
25,ETTm2,reg3,0.001,0.0,0,0,1,0.179384,0.247061,0.310937,0.405578
26,ETTm2,reg3,0.001,0.0,0,1,1,0.179959,0.246478,0.309623,0.404962
27,ETTm2,reg3,0.001,0.0,1,1,1,0.177449,0.246952,0.310139,0.412875
28,ETTm2,reg3,0.010,0.0,0,0,1,0.179121,0.247319,0.310977,0.405636
29,ETTm2,reg3,0.010,0.0,0,1,1,0.179116,0.246613,0.309648,0.405021
30,ETTm2,reg3,0.010,0.0,1,1,1,0.178298,0.247897,0.310601,0.406902
31,ETTm2,reg3,0.100,0.0,0,0,1,0.179123,0.247288,0.310991,0.405629


In [59]:
ettm2 = df.groupby(['reg','mask','conv','TD1','TD2']).min()[[96,192,336,720]]
ettm2.loc['base'] = [0.179,0.250,0.312,0.411]
ettm2.loc['repr'] = [0.182,0.252,0.313,0.413]

ettm2['avg'] = ettm2.mean(axis=1)
ettm2.sort_values('avg').round(3)

L,96,192,336,720,avg
"(reg3, 0.0, 0, 1, 1)",0.179,0.246,0.310,0.404,0.285
"(reg4, 0.0, 0, 1, 1)",0.179,0.247,0.310,0.404,0.285
"(reg3, 0.0, 1, 1, 1)",0.177,0.247,0.310,0.407,0.285
"(reg4, 0.0, 0, 0, 1)",0.179,0.247,0.310,0.405,0.285
"(reg3, 0.0, 0, 0, 1)",0.179,0.247,0.311,0.405,0.286
"(reg4, 0.0, 1, 1, 1)",0.178,0.248,0.311,0.407,0.286
base,0.179,0.250,0.312,0.411,0.288
repr,0.182,0.252,0.313,0.413,0.290


# Exchange:끝

In [60]:
df = get_result(DATASET='Exchange')
#df['avg'] = df.iloc[:,-4:].mean(axis=1)
#df[(df['reg']=='reg3')&(df['lamb']==0)&(df['conv']==0)&(df['TD1']==1)&(df['TD2']==1)].round(3)
#df[(df['reg']=='reg4')&(df['lamb']==0)&(df['conv']==0)&(df['TD1']==1)&(df['TD2']==1)].round(3)
df

L,data,reg,lamb,mask,conv,TD1,TD2,96,192,336,720
12,Exchange,reg3,0.0000,0.0,0,0,1,0.086257,0.179098,0.329639,0.853478
13,Exchange,reg3,0.0000,0.0,0,1,0,0.087421,0.180510,0.330409,0.857456
14,Exchange,reg3,0.0000,0.0,0,1,1,0.087696,0.180354,0.331563,0.859750
15,Exchange,reg3,0.0000,0.0,1,1,1,0.085893,0.179632,0.330319,0.857946
16,Exchange,reg3,0.0001,0.0,0,0,1,0.086029,0.178877,0.329642,0.853405
17,Exchange,reg3,0.0001,0.0,0,1,0,0.087027,0.180389,0.330380,0.857457
18,Exchange,reg3,0.0001,0.0,0,1,1,0.087644,0.180224,0.331564,0.859843
19,Exchange,reg3,0.0001,0.0,1,1,1,0.085756,0.179351,0.330338,0.857633
20,Exchange,reg3,0.0010,0.0,0,0,1,0.085589,0.179122,0.329766,0.853626
21,Exchange,reg3,0.0010,0.0,0,1,0,0.086168,0.179805,0.330396,0.857103


In [47]:
exchange = df.groupby(['reg','mask','conv','TD1','TD2']).min()[[96,192,336,720]]
exchange.loc['base'] = [0.086,0.182,0.332,0.867]
exchange.loc['repr'] = [0.086,0.182,0.331,0.858]
exchange['avg'] = exchange.mean(axis=1)
exchange.sort_values('avg').round(3)

L,96,192,336,720,avg
"(reg4, 0.0, 0, 0, 1)",0.085,0.178,0.328,0.849,0.360
"(reg4, 0.0, 0, 1, 0)",0.085,0.178,0.329,0.850,0.361
"(reg4, 0.0, 1, 1, 1)",0.085,0.178,0.330,0.854,0.361
"(reg3, 0.0, 0, 0, 1)",0.086,0.179,0.330,0.852,0.361
"(reg3, 0.0, 1, 1, 1)",0.084,0.178,0.330,0.855,0.362
"(reg3, 0.0, 0, 1, 0)",0.086,0.180,0.330,0.854,0.362
"(reg4, 0.0, 0, 1, 1)",0.087,0.179,0.331,0.857,0.364
repr,0.086,0.182,0.331,0.858,0.364
"(reg3, 0.0, 0, 1, 1)",0.087,0.180,0.332,0.859,0.364
base,0.086,0.182,0.332,0.867,0.367


# Electricity:끝

In [63]:
df = get_result(DATASET='ECL')
#df['avg'] = df.iloc[:,-4:].mean(axis=1)
##df[(df['reg']=='reg3')&(df['lamb']==0)&(df['conv']==0)&(df['TD1']==1)&(df['TD2']==1)].round(3)
#df[(df['reg']=='reg4')&(df['lamb']==0)&(df['conv']==0)&(df['TD1']==1)&(df['TD2']==1)].round(3)
df

L,data,reg,lamb,mask,conv,TD1,TD2,96,192,336,720
7,ECL,reg3,0.00000,0.0,0,1,1,0.140577,0.158071,0.176756,0.200767
8,ECL,reg3,0.00000,0.0,1,1,1,0.140307,0.166016,0.188246,0.204360
9,ECL,reg3,0.00001,0.0,0,1,1,0.139218,0.160311,0.177112,0.220369
10,ECL,reg3,0.00001,0.0,1,1,1,0.140336,0.163813,0.180960,0.206525
11,ECL,reg3,0.00010,0.0,0,0,1,NaN,NaN,NaN,0.220525
12,ECL,reg3,0.00010,0.0,0,1,0,0.140786,0.161007,0.179012,0.199768
13,ECL,reg3,0.00010,0.0,1,1,1,0.141148,0.163226,0.181862,0.208554
14,ECL,reg3,0.00100,0.0,0,1,0,0.169412,0.159976,0.177004,0.209062
15,ECL,reg3,0.00100,0.0,1,1,1,0.143103,0.160167,0.184371,0.210105
16,ECL,reg3,0.01000,0.0,0,1,0,0.180891,0.172180,0.188864,0.228719


In [64]:
ecl = df.groupby(['reg','mask','conv','TD1','TD2']).min()[[96,192,336,720]]
ecl.loc['base'] = [0.139,0.159,0.176,0.204]
ecl.loc['repr'] = [0.139,0.165,0.177,0.214]
ecl['avg'] = ecl.mean(axis=1)
ecl.sort_values('avg').round(3)

L,96,192,336,720,avg
"(reg4, 0.0, 0, 0, 1)",NaN,0.164,NaN,NaN,0.164
"(reg3, 0.0, 0, 1, 1)",0.139,0.158,0.177,0.201,0.169
"(reg3, 0.0, 0, 1, 0)",0.141,0.160,0.177,0.200,0.169
base,0.139,0.159,0.176,0.204,0.170
"(reg4, 0.0, 0, 1, 1)",0.142,0.159,0.175,0.205,0.170
"(reg3, 0.0, 1, 1, 1)",0.140,0.160,0.181,0.204,0.171
"(reg4, 0.0, 1, 1, 1)",0.144,0.161,0.177,0.211,0.173
repr,0.139,0.165,0.177,0.214,0.174
"(reg4, 0.0, 0, 1, 0)",0.145,0.159,0.177,0.224,0.176
"(reg3, 0.0, 0, 0, 1)",NaN,NaN,NaN,0.221,0.221


# Traffic

In [65]:
df = get_result(DATASET='traffic')
df['avg'] = df.iloc[:,-4:].mean(axis=1)
#df[(df['reg']=='reg3')&(df['lamb']==0)&(df['conv']==0)&(df['TD1']==1)&(df['TD2']==1)].round(3)
#df[(df['reg']=='reg4')&(df['lamb']==0)&(df['conv']==0)&(df['TD1']==1)&(df['TD2']==1)].round(3)
df

L,data,reg,lamb,mask,conv,TD1,TD2,96,192,336,720,avg
1,traffic,reg3,0.00000,0.0,0,1,1,0.378223,0.406840,0.500342,0.455693,0.435274
2,traffic,reg3,0.00000,0.0,1,1,1,0.382479,0.413353,0.415231,0.461808,0.418218
3,traffic,reg3,0.00001,0.0,0,1,1,0.381898,0.411846,0.485708,0.458410,0.434466
4,traffic,reg3,0.00001,0.0,1,1,1,0.382611,NaN,NaN,NaN,0.382611
5,traffic,reg3,0.00010,0.0,0,1,1,0.379015,0.410296,0.497404,0.455798,0.435628
6,traffic,reg3,0.00010,0.0,1,1,1,0.382164,NaN,NaN,NaN,0.382164
7,traffic,reg3,0.00100,0.0,0,1,1,0.382881,0.399321,0.416170,0.456157,0.413632
8,traffic,reg3,0.00100,0.0,1,1,1,0.382721,NaN,NaN,NaN,0.382721
9,traffic,reg3,0.01000,0.0,1,1,1,0.397432,NaN,NaN,NaN,0.397432
10,traffic,reg3,0.10000,0.0,1,1,1,0.430824,NaN,NaN,NaN,0.430824


In [29]:
traffic = df.groupby(['reg','mask','conv','TD1','TD2']).min()[[96,192,336,720]]
traffic.loc['base'] = [0.382,0.396,0.417,0.460]
traffic.loc['repr'] = [0.379,0.409,0.418,0.461]
traffic['avg'] = traffic.mean(axis=1)
traffic.sort_values('avg').round(3)

L,96,192,336,720,avg
base,0.382,0.396,0.417,0.460,0.414
repr,0.379,0.409,0.418,0.461,0.417
"(reg3, 0.0, 1, 1, 1)",0.382,0.413,0.415,0.462,0.418
"(reg4, 0.0, 1, 1, 1)",0.389,0.411,0.427,0.462,0.422


# Weather:끝

In [66]:
df = get_result(DATASET='weather')
#df['avg'] = df.iloc[:,-4:].mean(axis=1)
#df[(df['reg']=='reg3')&(df['lamb']==0)&(df['conv']==0)&(df['TD1']==1)&(df['TD2']==1)].round(3)
#df[(df['reg']=='reg4')&(df['lamb']==0)&(df['conv']==0)&(df['TD1']==1)&(df['TD2']==1)].round(3)
df

L,data,reg,lamb,mask,conv,TD1,TD2,96,192,336,720
11,weather,reg3,0.00000,0.0,1,1,1,0.172437,0.218870,0.275262,0.354263
12,weather,reg3,0.00001,0.0,1,1,1,0.172501,0.218964,0.275808,0.353772
13,weather,reg3,0.00010,0.0,1,1,1,0.172601,0.219113,0.274965,0.352851
14,weather,reg3,0.00100,0.0,1,1,1,0.172253,0.221015,0.275639,0.354464
15,weather,reg3,0.01000,0.0,1,1,1,0.177705,0.228039,0.283312,0.360263
16,weather,reg3,0.10000,0.0,1,1,1,0.181943,0.230394,0.284075,0.360440
17,weather,reg3,0.20000,0.0,1,1,1,0.182196,0.230686,0.284377,0.360328
18,weather,reg4,0.00010,0.0,1,1,1,0.173029,0.221201,0.277354,0.356294
19,weather,reg4,0.00100,0.0,1,1,1,0.173602,0.221062,0.276074,0.355045
20,weather,reg4,0.01000,0.0,1,1,1,0.180756,0.230677,0.284356,0.360637


In [32]:
weather = df.groupby(['reg','mask','conv','TD1','TD2']).min()[[96,192,336,720]]
weather.loc['base'] = [0.165,0.214,0.274,0.350]
weather.loc['repr'] = [0.165,0.215,0.273,0.353]

weather['avg'] = weather.mean(axis=1)
weather.sort_values('avg').round(3)

L,96,192,336,720,avg
base,0.165,0.214,0.274,0.350,0.251
repr,0.165,0.215,0.273,0.353,0.252
"(reg3, 0.0, 1, 1, 1)",0.172,0.219,0.275,0.353,0.255
"(reg4, 0.0, 1, 1, 1)",0.173,0.221,0.276,0.355,0.256


# PEMS03

In [75]:
df = get_result(DATASET='PEMS03')
df['avg'] = df.iloc[:,-4:].mean(axis=1)
#df[(df['reg']=='reg3')&(df['lamb']==0)&(df['conv']==0)&(df['TD1']==1)&(df['TD2']==1)].round(3)
#df[(df['reg']=='reg4')&(df['lamb']==0)&(df['conv']==0)&(df['TD1']==1)&(df['TD2']==1)].round(3)
df

L,data,reg,lamb,mask,conv,TD1,TD2,12,24,48,96,avg
0,PEMS03,reg3,0.0000,0.0,1,1,1,0.066354,0.090002,0.168946,0.233092,0.139598
1,PEMS03,reg3,0.0001,0.0,0,0,1,NaN,NaN,0.174798,0.307755,0.241277
2,PEMS03,reg3,0.0001,0.0,1,1,1,0.065734,0.089528,0.169128,0.233509,0.139475
3,PEMS03,reg3,0.0010,0.0,0,0,1,NaN,NaN,0.177218,NaN,0.177218
4,PEMS03,reg3,0.0010,0.0,1,1,1,0.066712,0.091037,0.171987,NaN,0.109912
5,PEMS03,reg3,0.0100,0.0,0,0,1,NaN,NaN,0.217713,NaN,0.217713
6,PEMS03,reg4,0.0000,0.0,1,1,1,0.066529,0.089355,0.171963,0.548403,0.219062
7,PEMS03,reg4,0.0001,0.0,1,1,1,0.065991,0.089599,0.173156,0.228339,0.139271
8,PEMS03,reg4,0.0010,0.0,1,1,1,0.067116,0.091250,0.178545,0.215833,0.138186
9,PEMS03,reg4,0.0100,0.0,1,1,1,0.069094,NaN,NaN,NaN,0.069094


In [76]:
pems03 = df.groupby(['reg','mask','conv','TD1','TD2']).min()[[12,24,48,96]]
pems03.loc['base'] = [0.065,0.087,0.133,0.201]
pems03.loc['repr'] = [0.066,0.088,0.165,0.213]

pems03['avg'] = pems03.mean(axis=1)
pems03.sort_values('avg').round(3)

L,12,24,48,96,avg
base,0.065,0.087,0.133,0.201,0.122
repr,0.066,0.088,0.165,0.213,0.133
"(reg4, 0.0, 1, 1, 1)",0.066,0.089,0.172,0.216,0.136
"(reg3, 0.0, 1, 1, 1)",0.066,0.090,0.169,0.233,0.139
"(reg3, 0.0, 0, 0, 1)",NaN,NaN,0.175,0.308,0.241


# PEMS04

In [77]:
df = get_result(DATASET='PEMS04')
df['avg'] = df.iloc[:,-4:].mean(axis=1)
df[(df['reg']=='reg3')&(df['lamb']==0)].round(3)
#df[(df['reg']=='reg4')&(df['lamb']==0)].round(3)


L,data,reg,lamb,mask,conv,TD1,TD2,12,24,48,96,avg
0,PEMS04,reg3,0.0,0.0,1,1,1,0.075,0.087,0.107,0.14,0.102


In [78]:
pems04 = df.groupby(['reg','mask','conv','TD1','TD2']).min()[[12,24,48,96]]
pems04.loc['base'] = [0.076,0.084,0.115,0.137]
pems04.loc['repr'] = [0.073,0.084,0.101,0.125]

pems04['avg'] = pems04.mean(axis=1) 
pems04.sort_values('avg').round(3)

L,12,24,48,96,avg
"(reg4, 0.0, 1, 1, 1)",0.075,0.087,NaN,NaN,0.081
repr,0.073,0.084,0.101,0.125,0.096
"(reg3, 0.0, 1, 1, 1)",0.075,0.087,0.107,0.136,0.101
base,0.076,0.084,0.115,0.137,0.103
"(reg3, 0.0, 0, 0, 1)",0.148,NaN,NaN,NaN,0.148


# PEMS07

In [79]:
df = get_result(DATASET='PEMS07')
df['avg'] = df.iloc[:,-4:].mean(axis=1)
df[(df['reg']=='reg3')&(df['lamb']==0)].round(3)
#df[(df['reg']=='reg4')&(df['lamb']==0)].round(3)
df

L,data,reg,lamb,mask,conv,TD1,TD2,12,24,48,avg
0,PEMS07,reg3,0.0000,0.0,1,1,1,0.060373,0.083557,0.109756,0.313422
1,PEMS07,reg3,0.0001,0.0,1,1,1,0.061012,NaN,NaN,0.530506


In [80]:
pems07 = df.groupby(['reg','mask','conv','TD1','TD2']).min()[[12,24,48,96]]
pems07.loc['base'] = [0.063,0.081,0.093,0.117]
pems07.loc['repr'] = [0.060,0.082,0.100,0.117]

pems07['avg'] = pems07.mean(axis=1)
pems07.sort_values('avg').round(3)

KeyError: '[96] not in index'

# PEMS08

In [81]:
df = get_result(DATASET='PEMS08')
df['avg'] = df.iloc[:,-4:].mean(axis=1)
#df[(df['reg']=='reg3')&(df['lamb']==0)&(df['conv']==0)&(df['TD1']==1)&(df['TD2']==1)].round(3)
#df[(df['reg']=='reg4')&(df['lamb']==0)&(df['conv']==0)&(df['TD1']==1)&(df['TD2']==1)].round(3)

In [82]:
pems08 = df.groupby(['reg','mask','conv','TD1','TD2']).min()[[12,24,48,96]]
pems08.loc['base'] = [0.076,0.104,0.167,0.245]
pems08.loc['repr'] = [0.076,0.109,0.173,0.271]

pems08['avg'] = pems08.mean(axis=1)
pems08.sort_values('avg').round(3)

L,12,24,48,96,avg
base,0.076,0.104,0.167,0.245,0.148
repr,0.076,0.109,0.173,0.271,0.157
"(reg3, 0.0, 1, 1, 1)",0.077,0.111,0.167,0.283,0.160
"(reg4, 0.0, 1, 1, 1)",0.079,0.116,0.178,0.273,0.162


In [96]:
etth1.index = etth1.index.astype('str')
etth2.index = etth2.index.astype('str')
ettm1.index = ettm1.index.astype('str')
ettm2.index = ettm2.index.astype('str')
exchange.index = exchange.index.astype('str')
ecl.index = ecl.index.astype('str')
weather.index = weather.index.astype('str')
traffic.index = traffic.index.astype('str')
pems03.index = pems03.index.astype('str')
#pems04.index = pems04.index.astype('str')
#pems07.index = pems07.index.astype('str')
pems08.index = pems08.index.astype('str')

In [100]:
etth1 = etth1.loc[["('reg3', 0.0, 0)","('reg4', 0.0, 0)",'base','repr']]
etth2 = etth2.loc[["('reg3', 0.0, 0)","('reg4', 0.0, 0)",'base','repr']]
ettm1 = ettm1.loc[["('reg3', 0.0, 0)","('reg4', 0.0, 0)",'base','repr']]
ettm2 = ettm2.loc[["('reg3', 0.0, 0)","('reg4', 0.0, 0)",'base','repr']]
exchange = exchange.loc[["('reg3', 0.0, 0)","('reg4', 0.0, 0)",'base','repr']]
ecl = ecl.loc[["('reg3', 0.0, 0)","('reg4', 0.0, 0)",'base','repr']]
weather = weather.loc[["('reg3', 0.0, 0)","('reg4', 0.0, 0)",'base','repr']]
traffic = traffic.loc[["('reg3', 0.0, 0)","('reg4', 0.0, 0)",'base','repr']]
pems03 = pems03.loc[["('reg3', 0.0, 0)","('reg4', 0.0, 0)",'base','repr']]
#pems04 = pems04.loc[["('reg3', 0.0, 0)","('reg4', 0.0, 0)",'base','repr']]
#pems07 = pems07.loc[["('reg3', 0.0, 0)","('reg4', 0.0, 0)",'base','repr']]
#pems08 = pems08.loc[["('reg3', 0.0, 0)","('reg4', 0.0, 0)",'base','repr']]

In [101]:
etth1.index = ['add2','add1','paper','reproduced']
etth2.index = ['add2','add1','paper','reproduced']
ettm1.index = ['add2','add1','paper','reproduced']
ettm2.index = ['add2','add1','paper','reproduced']
exchange.index = ['add2','add1','paper','reproduced']
ecl.index = ['add2','add1','paper','reproduced']
weather.index = ['add2','add1','paper','reproduced']
traffic.index = ['add2','add1','paper','reproduced']
pems03.index = ['add2','add1','paper','reproduced']
#pems04.index = ['add2','add1','paper','reproduced']
#pems07.index = ['add2','add1','paper','reproduced']
#pems08.index = ['add2','add1','paper','reproduced']


In [108]:
etth1.index.name = 'etth1'
etth2.index.name = 'etth2'
ettm1.index.name = 'ettm1'
ettm2.index.name = 'ettm2'
exchange.index.name = 'exchange'
ecl.index.name = 'ecl'
weather.index.name = 'weather'
traffic.index.name = 'traffic'
pems03.index.name = 'pems03'
#pems04.index.name = 'pems04'
#pems07.index.name = 'pems07'
#pems08.index.name = 'pems08'

In [111]:
display(etth1.round(3))
display(etth2.round(3))
display(ettm1.round(3))
display(ettm2.round(3))
display(exchange.round(3))
display(ecl.round(3))
display(weather.round(3))
#display(traffic.round(3))
#display(pems03.round(3))

L,96,192,336,720,avg
etth1,,,,,
add2,0.384,0.436,0.483,0.494,0.449
add1,0.384,0.436,0.483,0.496,0.450
paper,0.386,0.443,0.489,0.502,0.455
reproduced,0.385,0.445,0.490,0.506,0.456


L,96,192,336,720,avg
etth2,,,,,
add2,0.296,0.377,0.424,0.424,0.380
add1,0.297,0.378,0.425,0.424,0.381
paper,0.296,0.376,0.424,0.426,0.380
reproduced,0.297,0.378,0.425,0.432,0.383


L,96,192,336,720,avg
ettm1,,,,,
add2,0.329,0.375,0.409,0.472,0.396
add1,0.325,0.372,0.404,0.467,0.392
paper,0.333,0.376,0.408,0.475,0.398
reproduced,0.331,0.378,0.410,0.474,0.398


L,96,192,336,720,avg
ettm2,,,,,
add2,0.177,0.247,0.310,0.407,0.285
add1,0.178,0.248,0.311,0.407,0.286
paper,0.179,0.250,0.312,0.411,0.288
reproduced,0.182,0.252,0.313,0.413,0.290


L,96,192,336,720,avg
exchange,,,,,
add2,0.084,0.178,0.330,0.855,0.362
add1,0.085,0.178,0.330,0.854,0.361
paper,0.086,0.182,0.332,0.867,0.367
reproduced,0.086,0.182,0.331,0.858,0.364


L,96,192,336,720,avg
ecl,,,,,
add2,0.140,0.160,0.181,0.204,0.171
add1,0.144,0.161,0.177,0.211,0.173
paper,0.139,0.159,0.176,0.204,0.170
reproduced,0.139,0.165,0.177,0.214,0.174


L,96,192,336,720,avg
weather,,,,,
add2,0.172,0.219,0.275,0.353,0.255
add1,0.173,0.221,0.276,0.355,0.256
paper,0.165,0.214,0.274,0.350,0.251
reproduced,0.165,0.215,0.273,0.353,0.252
